# Diffusers for ecomerce product images



### 1. Enviroment setup

In [23]:
import os
from pathlib import Path

import torch
from PIL import Image
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from diffusers import EulerAncestralDiscreteScheduler

In [24]:
OUT = Path("output")
OUT.mkdir(exist_ok=True)

#Select Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16
DEVICE,dtype

(device(type='cuda'), torch.float16)

Load model & set Euler Ancestral Scheduler


In [25]:
model = "stabilityai/stable-diffusion-2-1-base"

#Load pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    model,
    torch_dtype = dtype,
    safety_checker = None
).to(DEVICE)


pipe.scheduler = EulerAncestralDiscreteScheduler.from_pretrained(model,subfolder = "scheduler")

pipe.enable_attention_slicing() #slices computing attention, saves memory

pipe.enable_attention_slicing()
try:
    pipe.enable_vae_slicing()
except Exception:
    pass

print("Pipeline ready on:", DEVICE)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Pipeline ready on: cuda


Text to Image generation

In [ ]:
prompt = "A high resolution product photo of a pair of white sneakers, space silver, seamless background, studio lighting, soft shadows, high detail "
negative_prompt = "low quality,blurry,watermark,text,logo,deformed"

image = pipe(
    prompt = prompt,
    negative_prompt = negative_prompt,
    width = 512,
    height = 512,
    num_inference_steps = 50,
    guidance_scale = 7.0
).images[0]


path = OUT / "phone_baseline.png"
image.save(path)
path

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
Img = Image.open(path)
Img.show()

In [ ]:
prompts = [
    "A luxury leather handbag with gold accents, product photo, white background, studio lighting, soft shadows, high detail",
    "A stainless steel wristwatch, macro product photo, white background, dramatic studio lighting, crisp edges, high detail",
    "A matte black wireless earbud charging case, product photo, white background, soft light, minimalistic, high detail",
]


paths = []
for i, p in enumerate(prompts,start = 1):
  img = pipe(
      prompt = p,
      negative_prompt = negative_prompt,
      width = 512,
      height = 512,
      num_inference_steps = 50,
      guidance_scale = 7.0
  ).images[0]

  path = OUT/f"product_{i}.png"
  img.save(path)
  paths.append(path)

paths


In [ ]:
from diffusers import DDIMScheduler

# Swap scheduler
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

prompt_ddim = "A premium insulated stainless steel water bottle, studio product photo, white background, soft shadow, high detail"

img_ddim = pipe(
    prompt=prompt_ddim,
    negative_prompt=negative_prompt,
    num_inference_steps=30,
    guidance_scale=7.0,
    height=512,
    width=512
).images[0]

p_ddim = OUT / "water_bottle_ddim.png"
img_ddim.save(p_ddim)
p_ddim


## 7) Image-to-Image (basic)
- Put a base image at `assets/base_product.jpg` (your own product photo).
- Change color/pattern with a simple prompt.


In [ ]:
BASE_IMG_PATH = "output/phone_baseline.png"  # replace with a real path you have

if os.path.exists(BASE_IMG_PATH):
    img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
        model, torch_dtype=dtype, safety_checker=None
    ).to(DEVICE)
    try:
        img2img.enable_attention_slicing()
    except Exception:
        pass

    init_image = Image.open(BASE_IMG_PATH).convert("RGB").resize((512, 512))

    prompt_i2i = "Same product but in navy blue, subtle geometric pattern, white background, studio lighting, product photo, high detail"

    img_var = img2img(
        prompt=prompt_i2i,
        negative_prompt=negative_prompt,
        image=init_image,
        strength=0.55,              # higher = more change
        guidance_scale=7.0,
        num_inference_steps=35
    ).images[0]

    out_i2i = OUT / "img2img_variant.png"
    img_var.save(out_i2i)
    print("Saved:", out_i2i)
else:
    print(f"Base image not found: {BASE_IMG_PATH}")
